In [1]:
import os, sys
sys.path.append('..')

import torch
import torch.nn as nn

In [2]:
class TimeSeriesTransformerAutoencoder(nn.Module):
    def __init__(self, input_dim, model_dim, num_heads, num_layers, seq_length, dropout=0.1):
        super(TimeSeriesTransformerAutoencoder, self).__init__()
        self.embedding = nn.Linear(input_dim, model_dim)
        self.pos_encoder = nn.Parameter(torch.zeros(1, seq_length, model_dim))
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout),
            num_layers=num_layers
        )
        self.transformer_decoder = nn.TransformerDecoder(
            nn.TransformerDecoderLayer(d_model=model_dim, nhead=num_heads, dropout=dropout),
            num_layers=num_layers
        )
        self.fc_out = nn.Linear(model_dim, input_dim)

    def forward(self, x):
        x = self.embedding(x) + self.pos_encoder
        z = self.transformer_encoder(x)
        x_rec = self.transformer_decoder(z, z)
        return self.fc_out(x_rec)

In [4]:
from dataset.cycling_dataset import CyclingDataset
from torch.utils.data import DataLoader

train_dataset = CyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_train.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_train.h5',
)

test_dataset = CyclingDataset(
    file_path_acc='/Users/Aleyna/downloads/preprocessed/handlebar_acc_test.h5',
    file_path_gyro='/Users/Aleyna/downloads/preprocessed/handlebar_gyro_test.h5',
)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)